In [ ]:
import cv2 
from scipy.spatial import distance as dist
import playsound
import numpy as np
import dlib
import imutils
from imutils import face_utils
from pygame import mixer
import argparse
from imutils.video import VideoStream

###***** Uncomment for RasPi *****###

#from picamera2 import Picamera2


"""
import time
import face_recognition
from threading import Thread
tutorial link : https://youtu.be/u7_34o_rs8o?si=dbzFq0jbhUL2SVCG
"""

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


'\nimport time\nimport face_recognition\nfrom threading import Thread\ntutorial link : https://youtu.be/u7_34o_rs8o?si=dbzFq0jbhUL2SVCG\n'

In [2]:
# Caculating the eye aspect ratio
def EAR(eye):
    #Eye points coordinates X,Y
    A = dist.euclidean(eye[1] ,eye[5])
    B = dist.euclidean(eye[2] ,eye[4])
    C = dist.euclidean(eye[0] ,eye[3])
    ear = (A+B)/(2*C)
    return ear

In [3]:
threshold = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector() #more faster than the cascaded method & uses svm
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [8]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]
# Option 0
cap = cv2.VideoCapture( 0 ) #to select the first camera if multiple cameras exist

# OPTION 1
# ap = argparse.ArgumentParser()
# ap.add_argument("-w", "--webcam", type=int, default=0,
#                 help="index of webcam on system")
# args = vars(ap.parse_args())
# vs = VideoStream(src=args["webcam"]).start() 

# Option 2
# vs= VideoStream(usePiCamera=True).start()

# Option 3
# camera = Picamera2()
# camera_config = camera.create_preview_configuration(main={"size": (640, 480)})
# camera.configure(camera_config)
# camera.start()

# Option 4
# camera = Picamera2()
# camera.preview_configuration.main.size = (800,800)
# camera.preview_configuration.main.format = "RGB888"
# camera.preview_configuration.align()
# camera.configure("preview")
# camera.start()

flag = 0


In [6]:
#mixer Should be initialized to work
mixer.init()
mixer.music.load("music.wav")

In [39]:
while True:
    ret, frame=cap.read()
    if not ret:
        print("Frame not read")
        break
    
    
    # For both option 1 and 2 for Ras_PI
    # frame = vs.read()
    
    #For Option 3 and 4
    #frame= camera.capture_array()

    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray , 0)
    
    for subject in subjects:
        shape = predict(gray,subject)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart: lEnd]
        rightEye = shape[rStart: rEnd]
        
        #calculating Eye Aspect Ratio
        leftEAR = EAR(leftEye)
        rightEAR = EAR(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        
        #these next 4 lines for drawing a green line around the eye
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        #Drowsiness detection
        if ear <threshold :
            flag +=1
            print(flag)
            if flag >= frame_check:
                cv2.putText(frame, "************alert********" , 
                (10 , 30 ) ,cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255) , 2)
                
                cv2.putText(frame, "****************ALERT!****************", (10,325),
				cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                mixer.music.play()
        else: 
            flag = 0
    cv2.imshow("Frame" , frame)
    cv2.waitKey(1) #display every 1 mS1
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    
cv2.destroyAllWindows()
cap.release()